![CUDA](figures/CUDA_Logo.jpg)

---
## Requisitos previos

Para aprovechar al máximo este lab, ya debería disponer de los siguientes conocimientos:

- Declarar variables, escribir bucles y usar sentencias if / else en C.
- Definir e invocar funciones en C.
- Asignar matrices en C.

---
## Sistemas heterogéneos

Los *sistemas heterogéneos*, son aquellos compuestos por CPU y GPU. En estos sistemas que comunmente también se denomina acelerador a la GPU necesitan de un control por parte del host o CPU que, a su vez, lanzan funciones que se beneficiarán del paralelismo masivo proporcionado por las GPU. La información sobre la GPU se puede consultar con el comando de línea de comando `nvidia-smi` (*Systems Management Interface*). Ejecute el comando `nvidia-smi` ahora, mediante `CTRL` + `ENTER` en la celda de ejecución de código a continuación. Encontrará estas celdas a lo largo de este laboratorio cada vez que necesite ejecutar código. El resultado de ejecutar el comando se imprimirá justo debajo de la celda de ejecución del código después de que se ejecute el código. Después de ejecutar el bloque de ejecución de código inmediatamente debajo, busque y anote el nombre de la GPU en la salida.

In [ ]:
!nvidia-smi

---

## Escritura de código para la GPU

CUDA proporciona extensiones para muchos lenguajes de programación comunes, en el caso de este laboratorio se utilizará el estandard C/C++. Estas extensiones permiten a los desarrolladores ejecutar fácilmente funciones en su código fuente en una GPU.

A continuación se muestra un archivo `.cu` (`.cu` es la extensión de archivo para los programas acelerados por CUDA). Contiene dos funciones, la primera que se ejecutará en la CPU, la segunda que se ejecutará en la GPU. Dedique un poco de tiempo a identificar las diferencias entre las funciones, tanto en términos de cómo se definen como de cómo se invocan.


```cpp
void CPUFunction()
{
  printf("This function is defined to run on the CPU.\n");
}

__global__ void GPUFunction()
{
  printf("This function is defined to run on the GPU.\n");
}

int main()
{
  CPUFunction();

  GPUFunction<<<1, 1>>>();
  cudaDeviceSynchronize();
}
```

Aquí hay algunas líneas de código a destacar, así como algunos otros términos comunes que se usan en computación sobre GPUs:

`__global__ void GPUFunction()`
  - La palabra clave `__global__` indica que la siguiente función se ejecutará en la GPU y se puede invocar **globalmente**, lo que en este contexto significa su ejecución se ordena desde la CPU, es decir el **lanzamiento del kernel**.
  - A menudo, el código que se ejecuta en la CPU se denomina código de **host** y el código que se ejecuta en la GPU se denomina código de **device** o **dispositivo**.
  - Preste atención en el tipo de devolución `void`. Se requiere que las funciones definidas con la palabra clave `__global__` devuelvan el tipo `void`.

`FunciónGPU<<<1, 1>>>();`
  - Por lo general, cuando llamamos a una función para que se ejecute en la GPU, llamamos a esta función **kernel**.
  - Al iniciar un kernel, debemos proporcionar una **configuración de ejecución**, que se realiza mediante el uso de la sintaxis `<<< ... >>>` justo antes de pasar al kernel los argumentos esperados.
  - La configuración de ejecución permite a los programadores especificar la **jerarquía de subprocesos** para el lanzamiento de un kernel, que define la cantidad de agrupaciones de subprocesos (llamados **CUDA bloques**), así como cuántos **subprocesos** o **CUDA threads** a ejecutar en cada bloque. La configuración de ejecución se explorará extensamente más adelante en el laboratorio, pero por el momento, tenga en cuenta que el kernel se inicia con un bloque de subprocesos "1" (el primer argumento de configuración de ejecución) que contiene un subproceso "1" (el segundo argumento de configuración) .

`cudaDeviceSynchronize();`
  - A diferencia de gran parte del código C/C++, el lanzamiento de kernels es **asincrónico**: el código de la CPU continuará ejecutándose *sin esperar a que se complete el lanzamiento del kernel*.
  - La llamada a `cudaDeviceSynchronize`, hará que el código del host (CPU) espere hasta que se complete el código del dispositivo (GPU), y solo entonces reanudará la ejecución en la CPU.


---
### Compilación y ejecución de código CUDA

Esta sección contiene detalles sobre el comando `nvcc` que permite compilar y ejecutar su programa `.cu`.

La GPU de NVIDIA se puede usar mediante el [**NVIDIA CUDA Compiler**](http://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html) `nvcc`, que puede compilar aplicaciones en CUDA, tanto el host como el código del dispositivo. Más información del compilador `nvcc` se puede encontrar en [la documentación] (http://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html).

El uso de `nvcc` será muy parecido si has utilizado previamente un compilador como `gcc`. Compilar, por ejemplo, un archivo `some-CUDA.cu`, es simplemente escribir en la consola:

`nvcc -arch=sm_70 -o out some-CUDA.cu -run`
  - `nvcc` es el comando de línea de comando para usar el compilador `nvcc`.
  - Se pasa `some-CUDA.cu` como archivo a compilar.
  - El indicador `o` se usa para especificar el archivo de salida para el programa compilado.
  - El indicador `arch` indica para qué **arquitectura** se deben compilar los archivos. Para el caso presente, `sm_70` servirá para compilar específicamente para la GPU en la que se ejecuta este laboratorio, pero para aquellos interesados ​​un conocimiento más profundo, se puede consulte los documentos sobre el flag [`arch`] (http://docs. nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html#options-for-steering-gpu-code-generation), [características de arquitectura virtual](http://docs.nvidia.com/cuda/cuda -compiler-driver-nvcc/index.html#gpu-feature-list) y [funciones de GPU](http://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html#gpu-feature -lista).
  - Como una cuestión de conveniencia, proporcionar el indicador `run` ejecutará el binario compilado con éxito.

---
# Jerarquía de bloques e hilos 

## Lanzamiento de kernels paralelos

La configuración de ejecución permite a los programadores especificar detalles sobre el lanzamiento del kernel para que se ejecute en paralelo en múltiples GPU **subprocesos**. Más en concreto, la configuración de ejecución permite a los programadores especificar cuántos grupos de subprocesos, llamados **bloques de subprocesos**, o simplemente **bloques CUDA**, y cuántos subprocesos les gustaría que contuviera cada bloque de subprocesos. La sintaxis para esto es:

`<<< NÚMERO_DE_BLOQUES, NÚMERO_DE_HILOS_POR_BLOQUE>>>`

** El código del kernel es ejecutado por cada subproceso en cada bloque de subprocesos configurado cuando se inicia el kernel**.

Por lo tanto, bajo el supuesto de que se ha definido un kernel llamado `someKernel`, lo siguiente es cierto:
  - `someKernel<<<1, 1>>>()` está configurado para ejecutarse en un bloque de un solo subproceso que tiene un solo subproceso y, por lo tanto, se ejecutará solo una vez.
  - `someKernel<<<1, 10>>>()` está configurado para ejecutarse en un solo bloque de subprocesos que tiene 10 subprocesos y, por lo tanto, se ejecutará 10 veces.
  - `someKernel<<<10, 1>>>()` está configurado para ejecutarse en 10 bloques de subprocesos, cada uno de los cuales tiene un solo subproceso y, por lo tanto, se ejecutará 10 veces.
  - `someKernel<<<10, 10>>>()` está configurado para ejecutarse en 10 bloques de subprocesos, cada uno de los cuales tiene 10 subprocesos y, por lo tanto, se ejecutará 100 veces.

![Jerarquía de hilos y bloques CUDA](figures/cuda_blocks.png)

---
## Indices para hilos y bloques CUDA

A cada subproceso se le asigna un índice dentro de su bloque de subprocesos, que comienza en `0`. Además, a cada bloque se le asigna un índice, que comienza en `0`. Así como los subprocesos se agrupan en bloques de subprocesos, los bloques se agrupan en un **grid**, que es la entidad más alta en la jerarquía de subprocesos de CUDA. En resumen, los kernels CUDA se ejecutan en un grid de 1 o más bloques, y cada bloque contiene la misma cantidad de 1 o más subprocesos (hilos).

Los kernels CUDA tienen acceso a variables especiales que identifican tanto el índice del subproceso (dentro del bloque) que ejecuta el núcleo como el índice del bloque (dentro de la cuadrícula) en el que se encuentra el subproceso. Estas variables son `threadIdx.x` y `blockIdx.x` respectivamente.

---
## Paralelizando bucles

Los bucles for en las aplicaciones de CPU son objetivos para la aceleración: en lugar de ejecutar cada iteración del bucle en serie, cada iteración del bucle se puede ejecutar en paralelo. Considere el siguiente for y observe, aunque es obvio, la variable *i* controla cuántas veces se ejecutará el bucle:

```cpp
int N = 2<<20;
for (int i = 0; i < N; ++i)
{
  printf("%d\n", i);
}
```

Para paralelizar este bucle, se deben seguir 2 pasos:

- Se debe escribir un kernel para hacer el trabajo de una **única iteración del bucle**.
- Debido a que el kernel será independiente de otros kernels en ejecución, la configuración de ejecución debe ser tal que el kernel se ejecute la cantidad correcta de veces, por ejemplo, la cantidad de veces que se habría iterado el bucle.

---
## Asignación de memoria para acceder a la GPU y la CPU

Las versiones más recientes de CUDA (versión 6 y posteriores) han facilitado la asignación de memoria que está disponible tanto para el host de la CPU como para cualquier cantidad de dispositivos de GPU, y aunque existen muchas [técnicas intermedias y avanzadas] (http://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#memory-optimizations) para la gestión de memoria que reportará un rendimiento más óptimo, la gestión de memoria CUDA más cómoda para desarrolladores se basa en la idea de memoria unificada.

Para asignar y liberar memoria, y obtener un puntero al que se pueda hacer referencia tanto en el código del host como del dispositivo, reemplace las llamadas a `malloc` y `free` con `cudaMallocManaged` y `cudaFree` como en el siguiente ejemplo:


```cpp
// CPU-only

int N = 2<<20;
size_t size = N * sizeof(int);

int *a;
a = (int *)malloc(size);

// Use `a` in CPU-only program.

free(a);
```

```cpp
// Accelerated

int N = 2<<20;
size_t size = N * sizeof(int);

int *a;
// Note the address of `a` is passed as first argument.
cudaMallocManaged(&a, size);

// Use `a` on the CPU and/or on any GPU in the accelerated system.

cudaFree(a);
```

---
## Manejo de Errores

Como en cualquier aplicación, el manejo de errores en código CUDA acelerado es fundamental. Muchas, si no la mayoría de las funciones de CUDA (consulte, por ejemplo, las [funciones de administración de memoria](http://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY)) devuelven un valor de tipo `cudaError_t`, que se puede utilizar para comprobar si se ha producido o no un error al llamar a la función. Aquí hay un ejemplo donde se realiza el manejo de errores para una llamada a `cudaMallocManaged`:

```cpp
cudaError_t err;
err = cudaMallocManaged(&a, N)                    // Assume the existence of `a` and `N`.

if (err != cudaSuccess)                           // `cudaSuccess` is provided by CUDA.
{
  printf("Error: %s\n", cudaGetErrorString(err)); // `cudaGetErrorString` is provided by CUDA.
}
```

Los kernels que están definidos para devolver `void`, no devuelven un valor de tipo `cudaError_t`. Para comprobar si se producen errores en el momento del lanzamiento del kernel, por ejemplo, si la configuración de lanzamiento es errónea, CUDA proporciona la función `cudaGetLastError`, que devuelve un valor de tipo `cudaError_t`.

```cpp
/*
 * This launch should cause an error, but the kernel itself
 * cannot return it.
 */

someKernel<<<1, -1>>>();  // -1 is not a valid number of threads.

cudaError_t err;
err = cudaGetLastError(); // `cudaGetLastError` will return the error from above.
if (err != cudaSuccess)
{
  printf("Error: %s\n", cudaGetErrorString(err));
}
```

Finalmente, para detectar errores que ocurren de forma asíncrona, por ejemplo, durante la ejecución de un kernel asíncrono, es esencial verificar el estado devuelto por una llamada API de tiempo de ejecución de CUDA de sincronización posterior, como `cudaDeviceSynchronize`, que devolverá un error si uno de los núcleos lanzados anteriormente ha fallado.


---
### CUDA Error Handling Function

Puede ser útil crear una macro que envuelva las llamadas a funciones de CUDA para verificar errores. Aquí hay un ejemplo, siéntase libre de usarlo en los ejercicios restantes:

```cpp
#include <stdio.h>
#include <assert.h>

inline cudaError_t checkCuda(cudaError_t result)
{
  if (result != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime Error: %s\n", cudaGetErrorString(result));
    assert(result == cudaSuccess);
  }
  return result;
}

int main()
{

/*
 * The macro can be wrapped around any function returning
 * a value of type `cudaError_t`.
 */

  checkCuda( cudaDeviceSynchronize() )
}
```

# Ejemplo suma de vectores

In [ ]:
%%writefile main.cu
#include <stdio.h>
#include <stdlib.h>

#include <sys/time.h>

//CUDA
#include <cuda.h>

double wtime(void)
{
        static struct timeval   tv0;
        double time_;

        gettimeofday(&tv0,(struct timezone*)0);
        time_=(double)((tv0.tv_usec + (tv0.tv_sec)*1000000));
        return( time_/1000000);
}


void vecAdd(float* A, float* B, float* C,
   int n)
{
	int i;
	for (i = 0; i < n; i++)
		C[i] = A[i] + B[i];
}


__global__ 
void vecAdd_GPU(float* A, float* B, float* C,
   int n)
{
	int i;
	i = threadIdx.x + blockDim.x * blockIdx.x;
	if(i<n) 
		C[i] = A[i] + B[i];
}

int main(int argc, char *argv[])
{
	float *a, *b, *c, *c_host;
	float *a_GPU, *b_GPU, *c_GPU;

	int i, N;

	double t0, t1;


	if(argc>1) {
		N = atoi(argv[1]); printf("N=%i\n", N);
	} else {
		printf("Error!!!! \n ./exec number\n");
	return (0);
	}

	// Mallocs CPU
	a  = (float *)malloc(sizeof(float)*N);
	b  = (float *)malloc(sizeof(float)*N);
	c  = (float *)malloc(sizeof(float)*N);
	c_host  = (float *)malloc(sizeof(float)*N);
	for (i=0; i<N; i++){ a[i] = i-1; b[i] = i;}

	/*****************/
	/* Add Matrix CPU*/
	/*****************/
	t0 = wtime();
	vecAdd(a, b, c, N);
	t1 = wtime(); printf("Time CPU=%f\n", t1-t0);

	// Get device memory for A, B, C
	// copy A and B to device memory
	cudaMalloc((void **) &a_GPU, N*sizeof(float));
	cudaMemcpy(a_GPU, a, N*sizeof(float), cudaMemcpyHostToDevice);
	cudaMalloc((void **) &b_GPU, N*sizeof(float));
	cudaMemcpy(b_GPU, b, N*sizeof(float), cudaMemcpyHostToDevice);
	cudaMalloc((void **) &c_GPU, N*sizeof(float));

	// Kernel execution in device
	// (vector add in device)
	dim3 DimBlock(256); // 256 thread per block
	dim3 DimGrid(ceil(N/256.0)+1);
	t0 = wtime();
	vecAdd_GPU<<<DimGrid,DimBlock>>>(a_GPU, b_GPU, c_GPU, N);
	cudaThreadSynchronize();
	t1 = wtime(); printf("Time GPU=%f\n", t1-t0);

	// copy C to host memory
	cudaMemcpy(c_host, c_GPU, N*sizeof(float), cudaMemcpyDeviceToHost);

	/************/
	/* Results  */
	/************/
	for (i=0; i<N; i++)
		if(fabs(c[i]-c_host[i])>1e-5){
			printf("c!=c_host in (%i): ", i);
			printf("C[%i] = %f C_GPU[%i]=%f\n", i, c[i], i, c_host[i] );
		}

	/* Free CPU */
	free(a);
	free(b);
	free(c);
	free(c_host);

	cudaFree(a_GPU); cudaFree(b_GPU); cudaFree(c_GPU);

	return(1);
}

* Compilar con **nvcc**

In [ ]:
!nvcc -o main main.cu

* Ejecutar

In [ ]:
!./main 1024000